In [1]:
#imports
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst
import gdal
import ogr
import osr
import gdalnumeric
import gdalconst
import os
import glob

In [2]:
gdal.UseExceptions()    # Enable exceptions

In [3]:
#import dir
iDirname = 'C:\\Users\\rfern\\gdrive\\s2msi\\import\\'
# working dir
wDirname = 'C:\\Users\\rfern\\gdrive\\s2msi\\working\\'
#export dir
eDirname = 'C:\\Users\\rfern\\gdrive\\s2msi\\export\\'


In [4]:
#L2a filename and subset region in degrees
ccLat = 45.40;
ccLong = -75.67;
rocWidth = 0.1;
rocWin = [ccLong-rocWidth, ccLat+rocWidth, ccLong+rocWidth, ccLat-rocWidth]

In [9]:
# run sen2cor and find the result file
L1name = "S2B_MSIL1C_20180509T155859_N0206_R097_T18TVR_20180509T200612.SAFE"
cmd = "C:\\Users\\rfern\\gdrive\\s2msi\\Sen2Cor-02.08.00-win64\\L2A_Process.bat "+iDirname+L1name+" --output_dir "+wDirname
os.system(cmd)
L2name = glob.glob(wDirname+L1name[0:8]+'2A'+L1name[10:24]+'*.SAFE')
exportName = (L2name[0][len(wDirname):(len(L2name[0])-5)])
print(exportName)

C:\Users\rfern\gdrive\s2msi\Sen2Cor-02.08.00-win64\L2A_Process.bat C:\Users\rfern\gdrive\s2msi\import\S2B_MSIL1C_20180509T155859_N0206_R097_T18TVR_20180509T200612.SAFE --output_dir C:\Users\rfern\gdrive\s2msi\working\


0

C:\Users\rfern\gdrive\s2msi\working\S2B_MSIL2A_20180509T155859_N9999_R097_T18TVR_20200522T010435.SAFE
S2B_MSIL2A_20180509T155859_N9999_R097_T18TVR_20200522T010435


In [13]:
# Open L2a data 
ds = gdal.Open(L2name[0]+'\MTD_MSIL2A.xml')
#ds.GetMetadata('SUBDATASETS')

In [14]:
# Open L2a 10m dataset and print info
f10m = gdal.Open('SENTINEL2_L2A:'+L2name[0]+'\MTD_MSIL2A.xml'+':10m:EPSG_32618')
#gdal.Info(f10m)


In [15]:
# Open L2a 20m dataset and print info
f20m = gdal.Open('SENTINEL2_L2A:'+L2name[0]+'\MTD_MSIL2A.xml'+':20m:EPSG_32618')
#gdal.Info(f20m)

In [16]:
# Open L2a 60m dataset and print info
f60m = gdal.Open('SENTINEL2_L2A:'+L2name[0]+'\MTD_MSIL2A.xml'+':60m:EPSG_32618')
#gdal.Info(f20m)

In [17]:
# Export 10m version of L2A product (no ancillary bands) and output file describing contents
gdal.Translate(wDirname+'b4.tif',f10m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b3.tif',f10m,bandList=[2],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b2.tif',f10m,bandList=[3],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b8.tif',f10m,bandList=[4],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b5.tif',f20m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b6.tif',f20m,bandList=[2],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b7.tif',f20m,bandList=[3],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b8a.tif',f20m,bandList=[4],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b11.tif',f20m,bandList=[5],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b12.tif',f20m,bandList=[6],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b1.tif',f60m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(wDirname+'b9.tif',f60m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)

vrtOptions = gdal.BuildVRTOptions(separate=True)
gdal.BuildVRT(wDirname+'merged.vrt',[wDirname+'b1.tif',wDirname+'b2.tif',wDirname+'b3.tif',wDirname+'b4.tif', \
                                     wDirname+'b5.tif',wDirname+'b6.tif',wDirname+'b7.tif',wDirname+'b8.tif', \
                                     wDirname+'b8a.tif',wDirname+'b9.tif',wDirname+'b11.tif',wDirname+'b12.tif'] \
                                   ,options=vrtOptions)
gdal.Translate(eDirname+exportName+'.tif',wDirname+'merged.vrt')
gdal.Info(wDirname+'merged.vrt')

'Driver: VRT/Virtual Raster\nFiles: C:\\Users\\rfern\\gdrive\\s2msi\\working\\merged.vrt\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b1.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b2.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b3.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b4.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b5.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b6.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b7.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b8.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b8a.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b9.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b11.tif\n       C:\\Users\\rfern\\gdrive\\s2msi\\working\\b12.tif\nSize is 1548, 2237\nCoordinate System is:\nPROJCRS["WGS 84 / UTM zone 18N",\n    BASEGEOGCRS["WGS 84",\n        DATUM["World Geodetic System 1984",\n            ELLIPSOID["WGS 84",6378137,298.257223563,\n                LEN